In [1]:
import numpy as np
import pandas as pd
#Set the option to display all columns
#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import timeit
#run

In [2]:
#path=input("Enter File Name")
path='daily_sell_thru_forecast_20231121.xlsx'
#run

In [3]:
#df = pd.read_excel(path)
#df
xls = pd.ExcelFile(path)

# Get the sheet names from the ExcelFile object
sheet_names = xls.sheet_names
#run

In [4]:
sheet_names

['Sheet1']

In [5]:
# Create a dictionary to store DataFrames
sheet_data = {}

# Loop through the sheet names and read the data from each sheet into separate DataFrames
for sheet_name in sheet_names:
    df = pd.read_excel(xls, sheet_name)
    # Store the DataFrame in the dictionary with the sheet name as the key
    sheet_data[sheet_name] = df


In [ ]:
sheet_data

In [6]:
sheet_data.keys()

dict_keys(['Sheet1'])

In [7]:
sheet_data['Sheet1'].head()

,customer code,location code,item code,category,forecast date,2023-07-02 00:00:00,2023-07-03 00:00:00,2023-07-04 00:00:00,2023-07-05 00:00:00,2023-07-06 00:00:00,...,2024-06-20 00:00:00,2024-06-21 00:00:00,2024-06-22 00:00:00,2024-06-23 00:00:00,2024-06-24 00:00:00,2024-06-25 00:00:00,2024-06-26 00:00:00,2024-06-27 00:00:00,2024-06-28 00:00:00,2024-06-29 00:00:00
0,BKD,BKD0001,403026,CAT,2023-07-02,2.823586,3.445799,3.432877,3.390589,3.398773,...,2.431849,2.441294,2.614176,2.074078,2.531126,2.521635,2.490572,2.496584,2.506280,2.683764
1,BKD,BKD0001,401242,CAT,2023-07-02,2.031291,3.333810,3.180916,3.052708,3.074730,...,2.368687,2.261255,2.731000,1.625636,2.668038,2.545677,2.443073,2.460696,2.349092,2.837083
2,BKD,BKD0001,400717,CAT,2023-07-02,0.088600,0.145413,0.138744,0.133152,0.134113,...,0.103345,0.098658,0.119152,0.070900,0.116363,0.111026,0.106551,0.107320,0.102452,0.123735
3,BKD,BKD0001,401810,CAT,2023-07-02,0.012874,0.031337,0.030195,0.037663,0.022903,...,0.008977,0.014911,0.011185,0.011127,0.027085,0.026098,0.032553,0.019796,0.032881,0.024665
4,BKD,BKD0001,404413,CAT,2023-07-02,3.277009,5.151086,4.683148,4.742782,4.726064,...,3.640765,3.775160,3.568208,2.861001,4.497168,4.088633,4.140697,4.126101,4.278412,4.043872


In [8]:
sheet_data['Sheet1'].columns

Index([    'customer code',     'location code',         'item code',
                'category',     'forecast date', 2023-07-02 00:00:00,
       2023-07-03 00:00:00, 2023-07-04 00:00:00, 2023-07-05 00:00:00,
       2023-07-06 00:00:00,
       ...
       2024-06-20 00:00:00, 2024-06-21 00:00:00, 2024-06-22 00:00:00,
       2024-06-23 00:00:00, 2024-06-24 00:00:00, 2024-06-25 00:00:00,
       2024-06-26 00:00:00, 2024-06-27 00:00:00, 2024-06-28 00:00:00,
       2024-06-29 00:00:00],
      dtype='object', length=369)

In [9]:
final = {}
for month, df in sheet_data.items():
    melted_df = pd.melt(df, id_vars=['customer code', 'location code', 'item code', 'category', 'forecast date'], var_name='Date', value_name='Sales')
    final[month] = melted_df


In [ ]:
final

In [10]:
concatenated_dfs = []

for name, df in final.items():
    concatenated_dfs.append(df)

# Concatenate the DataFrames vertically
combined_df = pd.concat(concatenated_dfs, axis=0)

# Reset the index
combined_df = combined_df.reset_index(drop=True)

In [11]:
combined_df.head()

,customer code,location code,item code,category,forecast date,Date,Sales
0,BKD,BKD0001,403026,CAT,2023-07-02,2023-07-02 00:00:00,2.823586
1,BKD,BKD0001,401242,CAT,2023-07-02,2023-07-02 00:00:00,2.031291
2,BKD,BKD0001,400717,CAT,2023-07-02,2023-07-02 00:00:00,0.088600
3,BKD,BKD0001,401810,CAT,2023-07-02,2023-07-02 00:00:00,0.012874
4,BKD,BKD0001,404413,CAT,2023-07-02,2023-07-02 00:00:00,3.277009


In [12]:
combined_df['Date'] = pd.to_datetime(combined_df['Date']).dt.date

In [13]:
combined_df.head(100)

,customer code,location code,item code,category,forecast date,Date,Sales
0,BKD,BKD0001,403026,CAT,2023-07-02,2023-07-02,2.823586
1,BKD,BKD0001,401242,CAT,2023-07-02,2023-07-02,2.031291
2,BKD,BKD0001,400717,CAT,2023-07-02,2023-07-02,0.088600
3,BKD,BKD0001,401810,CAT,2023-07-02,2023-07-02,0.012874
4,BKD,BKD0001,404413,CAT,2023-07-02,2023-07-02,3.277009
5,BKD,BKD0001,404152,CAT,2023-07-02,2023-07-02,2.179053
6,BKD,BKD0001,401713,CAT,2023-07-02,2023-07-02,0.024648
7,BKD,BKD0001,404193,CAT,2023-07-02,2023-07-02,0.017713
8,BKD,BKD0001,400310,CAT,2023-07-02,2023-07-02,2.745539
9,BKD,BKD0001,405060,CAT,2023-07-02,2023-07-02,0.456812


In [14]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11340056 entries, 0 to 11340055
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   customer code  object        
 1   location code  object        
 2   item code      int64         
 3   category       object        
 4   forecast date  datetime64[ns]
 5   Date           object        
 6   Sales          float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 605.6+ MB


In [15]:
combined_df.to_parquet('daily_sell_thru_forecast_20231121.parquet',index=False)

C:\Users\Junaid Bilal\anaconda3\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
